In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Load the data
df = pd.read_csv('../data/movies_raw.csv')

print(f"Dataset shape: {df.shape}")
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")
df.head()

In [ ]:
# Check data types and missing values
print("=== Data Info ===")
print(df.info())

print("\n=== Missing Values ===")
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print(missing)

print("\n=== Column Names ===")
print(df.columns.tolist())

In [15]:
# Check if genres column exists and what it looks like
if 'genres' in df.columns:
    print("=== Genre Data Sample ===")
    print(df['genres'].head(10))
    print(f"\nData type: {df['genres'].dtype}")
    
    # Check if it's a string or list
    sample_genre = df['genres'].iloc[0]
    print(f"\nSample genre value: {sample_genre}")
    print(f"Type: {type(sample_genre)}")
else:
    print("No 'genres' column found in data")
    print("Available columns:", df.columns.tolist())

=== Genre Data Sample ===
0    [{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...
1    [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
2    [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...
3    [{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...
4    [{'id': 28, 'name': 'Action'}, {'id': 10752, '...
5    [{'id': 80, 'name': 'Crime'}, {'id': 9648, 'na...
6    [{'id': 16, 'name': 'Animation'}, {'id': 12, '...
7    [{'id': 9648, 'name': 'Mystery'}, {'id': 35, '...
8    [{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...
9    [{'id': 10749, 'name': 'Romance'}, {'id': 18, ...
Name: genres, dtype: object

Data type: object

Sample genre value: [{'id': 27, 'name': 'Horror'}, {'id': 53, 'name': 'Thriller'}]
Type: <class 'str'>


In [ ]:
# Key statistics for numeric columns
print("=== Revenue Statistics ===")
print(df['revenue'].describe())

print("\n=== Budget Statistics ===")
print(df['budget'].describe())

print("\n=== Basic Metrics ===")
print(f"Average Budget: ${df['budget'].mean():,.0f}")
print(f"Average Revenue: ${df['revenue'].mean():,.0f}")
print(f"Average Vote: {df['vote_average'].mean():.2f}")
print(f"Average Runtime: {df['runtime'].mean():.0f} minutes")

In [ ]:
# Movies with zero budget or revenue (data quality issues)
print("=== Data Quality Checks ===")
print(f"Movies with budget = 0: {(df['budget'] == 0).sum()}")
print(f"Movies with revenue = 0: {(df['revenue'] == 0).sum()}")
print(f"Movies with both = 0: {((df['budget'] == 0) & (df['revenue'] == 0)).sum()}")

# Remove movies with zero budget or revenue
df_clean = df[(df['budget'] > 0) & (df['revenue'] > 0)].copy()
print(f"\nRows before cleaning: {len(df)}")
print(f"Rows after cleaning: {len(df_clean)}")
print(f"Removed: {len(df) - len(df_clean)} movies")

In [16]:
import json
from ast import literal_eval

# Function to extract genre names
def extract_genre_names(genre_string):
    """Extract genre names from TMDB genre format"""
    if pd.isna(genre_string) or genre_string == '[]':
        return []
    
    try:
        # Convert string to list of dictionaries
        genres = literal_eval(genre_string)
        # Extract just the names
        return [g['name'] for g in genres]
    except:
        return []

# Apply to dataframe
df_clean['genre_list'] = df_clean['genres'].apply(extract_genre_names)

# See what genres we have
all_genres = []
for genres in df_clean['genre_list']:
    all_genres.extend(genres)

genre_counts = pd.Series(all_genres).value_counts()
print("=== Genre Distribution ===")
print(genre_counts)

# Extract first (primary) genre
df_clean['primary_genre'] = df_clean['genre_list'].apply(lambda x: x[0] if len(x) > 0 else 'Unknown')

# Count genres per movie
df_clean['genre_count'] = df_clean['genre_list'].apply(len)

print(f"\n=== Genre Statistics ===")
print(f"Movies with genres: {(df_clean['genre_count'] > 0).sum()}")
print(f"Average genres per movie: {df_clean['genre_count'].mean():.1f}")
print(f"\nMost common primary genre: {df_clean['primary_genre'].value_counts().head()}")

=== Genre Distribution ===
Action             71
Adventure          57
Comedy             50
Thriller           48
Science Fiction    41
Horror             37
Drama              34
Fantasy            32
Family             30
Animation          26
Crime              20
Romance            13
Mystery             9
History             5
Music               4
War                 2
Western             1
Name: count, dtype: int64

=== Genre Statistics ===
Movies with genres: 159
Average genres per movie: 3.0

Most common primary genre: primary_genre
Action             45
Horror             29
Animation          14
Science Fiction    14
Drama              14
Name: count, dtype: int64


In [17]:
# Get the most common genres (we'll use top 10-15)
top_genres = genre_counts.head(15).index.tolist()
print(f"Using top {len(top_genres)} genres: {top_genres}")

# Create binary columns for each top genre
for genre in top_genres:
    df_clean[f'genre_{genre}'] = df_clean['genre_list'].apply(
        lambda x: 1 if genre in x else 0
    )

# Show genre columns created
genre_columns = [col for col in df_clean.columns if col.startswith('genre_')]
print(f"\n✓ Created {len(genre_columns)} genre features")
print(genre_columns)

# Example: Show movies with Action genre
print(f"\nMovies with Action: {df_clean['genre_Action'].sum()}")

Using top 15 genres: ['Action', 'Adventure', 'Comedy', 'Thriller', 'Science Fiction', 'Horror', 'Drama', 'Fantasy', 'Family', 'Animation', 'Crime', 'Romance', 'Mystery', 'History', 'Music']

✓ Created 17 genre features
['genre_list', 'genre_count', 'genre_Action', 'genre_Adventure', 'genre_Comedy', 'genre_Thriller', 'genre_Science Fiction', 'genre_Horror', 'genre_Drama', 'genre_Fantasy', 'genre_Family', 'genre_Animation', 'genre_Crime', 'genre_Romance', 'genre_Mystery', 'genre_History', 'genre_Music']

Movies with Action: 71


In [ ]:
# Average revenue by primary genre
plt.figure(figsize=(12, 6))
genre_revenue = df_clean.groupby('primary_genre')['revenue'].mean().sort_values(ascending=False)
genre_revenue.plot(kind='bar', color='teal', edgecolor='black')
plt.xlabel('Primary Genre')
plt.ylabel('Average Revenue ($)')
plt.title('Average Revenue by Primary Genre')
plt.xticks(rotation=45, ha='right')
plt.ticklabel_format(style='plain', axis='y')
plt.tight_layout()
plt.show()

print("Top revenue genres:")
print(genre_revenue.head(10))

In [ ]:
# Calculate profit and ROI
df_clean['profit'] = df_clean['revenue'] - df_clean['budget']
df_clean['roi'] = ((df_clean['revenue'] - df_clean['budget']) / df_clean['budget']) * 100

# Extract date features
df_clean['release_date'] = pd.to_datetime(df_clean['release_date'])
df_clean['release_year'] = df_clean['release_date'].dt.year
df_clean['release_month'] = df_clean['release_date'].dt.month
df_clean['release_day_of_week'] = df_clean['release_date'].dt.dayofweek  # 0=Monday, 6=Sunday

# Is it a summer blockbuster? (May-August)
df_clean['is_summer'] = df_clean['release_month'].isin([5, 6, 7, 8]).astype(int)

# Is it a holiday release? (November-December)
df_clean['is_holiday'] = df_clean['release_month'].isin([11, 12]).astype(int)

# Budget categories
df_clean['budget_category'] = pd.cut(df_clean['budget'], 
                                      bins=[0, 10_000_000, 50_000_000, 100_000_000, float('inf')],
                                      labels=['Low', 'Medium', 'High', 'Blockbuster'])

print("New features created:")
print(df_clean[['title', 'budget', 'revenue', 'profit', 'roi', 'release_month', 'is_summer']].head())

In [ ]:
# Revenue distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df_clean['revenue'], bins=50, edgecolor='black')
plt.xlabel('Revenue ($)')
plt.ylabel('Frequency')
plt.title('Revenue Distribution')

plt.subplot(1, 2, 2)
plt.hist(np.log10(df_clean['revenue']), bins=50, edgecolor='black', color='coral')
plt.xlabel('Revenue (log scale)')
plt.ylabel('Frequency')
plt.title('Revenue Distribution (Log Scale)')

plt.tight_layout()
plt.show()

print("Note: Revenue is heavily right-skewed. Most movies make modest amounts, few make billions.")

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df_clean['budget'], df_clean['revenue'], alpha=0.5)
plt.xlabel('Budget ($)')
plt.ylabel('Revenue ($)')
plt.title('Budget vs Revenue')

# Add diagonal line (break-even point)
max_val = max(df_clean['budget'].max(), df_clean['revenue'].max())
plt.plot([0, max_val], [0, max_val], 'r--', label='Break-even line')
plt.legend()
plt.ticklabel_format(style='plain', axis='both')
plt.tight_layout()
plt.show()

# Calculate how many movies were profitable
profitable = (df_clean['revenue'] > df_clean['budget']).sum()
print(f"\nProfitable movies: {profitable} ({profitable/len(df_clean)*100:.1f}%)")

In [ ]:
# Highest revenue movies
print("=== Top 10 Highest Revenue ===")
top_revenue = df_clean.nlargest(10, 'revenue')[['title', 'budget', 'revenue', 'profit', 'roi', 'release_year']]
print(top_revenue.to_string())

# Highest ROI movies
print("\n=== Top 10 Highest ROI ===")
top_roi = df_clean.nlargest(10, 'roi')[['title', 'budget', 'revenue', 'profit', 'roi', 'release_year']]
print(top_roi.to_string())

In [ ]:
# Revenue by release month
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
monthly_revenue = df_clean.groupby('release_month')['revenue'].mean()
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.bar(range(1, 13), monthly_revenue)
plt.xlabel('Month')
plt.ylabel('Average Revenue ($)')
plt.title('Average Revenue by Release Month')
plt.xticks(range(1, 13), months, rotation=45)

plt.subplot(1, 2, 2)
summer_vs_other = df_clean.groupby('is_summer')['revenue'].mean()
plt.bar(['Other Months', 'Summer'], summer_vs_other)
plt.ylabel('Average Revenue ($)')
plt.title('Summer vs Non-Summer Releases')

plt.tight_layout()
plt.show()

In [ ]:
# Revenue by budget category
plt.figure(figsize=(10, 6))
budget_analysis = df_clean.groupby('budget_category').agg({
    'revenue': 'mean',
    'roi': 'mean',
    'title': 'count'
}).round(2)
budget_analysis.columns = ['Avg Revenue', 'Avg ROI %', 'Count']
print(budget_analysis)

# Plot
budget_analysis['Avg Revenue'].plot(kind='bar', color='skyblue', edgecolor='black')
plt.ylabel('Average Revenue ($)')
plt.xlabel('Budget Category')
plt.title('Average Revenue by Budget Category')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Select numeric features for correlation
numeric_features = ['budget', 'revenue', 'profit', 'roi', 'runtime', 
                    'vote_average', 'vote_count', 'popularity', 
                    'release_year', 'release_month']

# Check which features exist in your data
available_features = [f for f in numeric_features if f in df_clean.columns]

# Correlation heatmap
plt.figure(figsize=(12, 10))
correlation = df_clean[available_features].corr()
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Feature Correlation Heatmap')
plt.tight_layout()
plt.show()

print("\nKey Correlations with Revenue:")
print(correlation['revenue'].sort_values(ascending=False))

In [ ]:
# Save the cleaned dataset
df_clean.to_csv('../data/movies_cleaned.csv', index=False)
print(f"✓ Cleaned data saved: {len(df_clean)} movies")
print(f"✓ File: data/movies_cleaned.csv")

# Summary stats
print("\n=== Final Dataset Summary ===")
print(f"Total movies: {len(df_clean)}")
print(f"Date range: {df_clean['release_year'].min()} - {df_clean['release_year'].max()}")
print(f"Average budget: ${df_clean['budget'].mean():,.0f}")
print(f"Average revenue: ${df_clean['revenue'].mean():,.0f}")
print(f"Average ROI: {df_clean['roi'].mean():.1f}%")